In [48]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [49]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging


In [50]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [51]:
mat_file_list= ['arrhythmia.mat',
               'Cardio.mat',
               'glass.mat',
               'ionosphere.mat',
               'letter.mat',
               'lympho.mat',
               'mnist.mat',
               'musk.mat',
               'optdigits.mat',
               'pendigits.mat',
               'pima.mat',
               'satellite.mat',
               'satimage-2.mat',
               'shuttle.mat',
               'vertebral.mat',
               'vowels.mat',
               'wbc.mat']

In [52]:
df_columns=['Data','#Samples','#Dimesnions','Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

In [53]:
roc_df=pd.DataFrame(columns=df_columns) #region of characteristics
prn_df=pd.DataFrame(columns=df_columns) #precision score
time_df=pd.DataFrame(columns=df_columns) #time difference dataframe

In [54]:
mat_file_list

['arrhythmia.mat',
 'Cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [55]:
from scipy.io import loadmat

In [85]:
data=loadmat('Downloads/pyod dataset/Dataset/Anamoly_detec_data/musk.mat')

In [86]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2015-07-05 13:16:13 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[  46., -108.,  -60., ...,  -50., -112.,   96.],
        [  41., -188., -145., ...,  -61., -136.,   79.],
        [  46., -194., -145., ...,  -67., -145.,   39.],
        ...,
        [  43.,   87.,   31., ...,  -67., -159.,  -22.],
        [  43.,   63.,   41., ...,  -69., -270.,   -5.],
        [  43.,   64.,   41., ...,  -69., -270.,   -6.]]),
 'y': array([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])}

In [87]:
len(data)

5

In [88]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [89]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2015-07-05 13:16:13 UTC', '1.0', [], array([[  46., -108.,  -60., ...,  -50., -112.,   96.],
       [  41., -188., -145., ...,  -61., -136.,   79.],
       [  46., -194., -145., ...,  -67., -145.,   39.],
       ...,
       [  43.,   87.,   31., ...,  -67., -159.,  -22.],
       [  43.,   63.,   41., ...,  -69., -270.,   -5.],
       [  43.,   64.,   41., ...,  -69., -270.,   -6.]]), array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])])

In [90]:
type(data['X']),data['X'].shape

(numpy.ndarray, (3062, 166))

In [91]:
type(data['y']),data['y'].shape 

(numpy.ndarray, (3062, 1))

In [63]:
roc_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [64]:
prn_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [65]:
time_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [92]:
from time import time
random_state=np.random.RandomState(42)
for mat_file in mat_file_list:
    print('\n...Processing',mat_file,'...')
    mat=loadmat(os.path.join('Downloads/pyod dataset/Dataset/Anamoly_detec_data',mat_file))
    
    X=mat['X']
    y=mat['y'].ravel()
    outliers_fraction=np.count_nonzero(y)/len(y)
    outliers_percentage=round(outliers_fraction*100,ndigits=4)
    
#construct containers for saving result
    roc_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    prn_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    time_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]

#60% data for training and 40% for testing
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=random_state)

#standardizing data
    X_train_norm,X_test_norm=standardizer(X_train,X_test)

    classifiers={'ABOD':ABOD(contamination=outliers_fraction),
            'CBLOF':CBLOF(contamination=outliers_fraction,check_estimator=False,random_state=random_state),
            'Feature Bagging':FeatureBagging(contamination=outliers_fraction,random_state=random_state),
            'HBOS':HBOS(contamination=outliers_fraction),
            'Isolation Forest':IForest(contamination=outliers_fraction,random_state=random_state),
            'KNN':KNN(contamination=outliers_fraction),
            'LOF':LOF(contamination=outliers_fraction),
            'MCD':MCD(contamination=outliers_fraction,random_state=random_state),
            'OCSVM':OCSVM(contamination=outliers_fraction),
            'PCA':PCA(contamination=outliers_fraction,random_state=random_state),}

    for clf_name,clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test_scores=clf.decision_function(X_test_norm)
        t1=time()
        duration=round(t1-t0,ndigits=4)
        time_list.append(duration)
        
        roc=round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn=round(precision_n_scores(y_test,test_scores),ndigits=4)
        
        print('{clf_name} ROC: {roc}, precision @ rank n: {prn},'
             'execution time: {duration}s'.format(clf_name=clf_name,roc=roc,prn=prn,duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df.columns=df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(roc_list).transpose()
    temp_df.columns=df_columns
    roc_df=pd.concat([roc_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(prn_list).transpose()
    temp_df.columns=df_columns
    prn_df=pd.concat([prn_df,temp_df],axis=0)


...Processing arrhythmia.mat ...
ABOD ROC: 0.7687, precision @ rank n: 0.3571,execution time: 0.21s
CBLOF ROC: 0.7684, precision @ rank n: 0.4643,execution time: 0.1185s


c:\python36\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Feature Bagging ROC: 0.7799, precision @ rank n: 0.5,execution time: 0.4758s
HBOS ROC: 0.8511, precision @ rank n: 0.5714,execution time: 0.106s
Isolation Forest ROC: 0.8478, precision @ rank n: 0.5357,execution time: 0.548s
KNN ROC: 0.782, precision @ rank n: 0.5,execution time: 0.071s
LOF ROC: 0.7787, precision @ rank n: 0.4643,execution time: 0.077s


c:\python36\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


MCD ROC: 0.8228, precision @ rank n: 0.4286,execution time: 1.2781s
OCSVM ROC: 0.7986, precision @ rank n: 0.5,execution time: 0.03s
PCA ROC: 0.7997, precision @ rank n: 0.5,execution time: 0.061s

...Processing Cardio.mat ...
ABOD ROC: 0.5763, precision @ rank n: 0.1875,execution time: 0.459s
CBLOF ROC: 0.8221, precision @ rank n: 0.4844,execution time: 0.176s

...


HBOS ROC: 0.2695, precision @ rank n: 0.0,execution time: 0.002s
Isolation Forest ROC: 0.3576, precision @ rank n: 0.0,execution time: 0.398s
KNN ROC: 0.318, precision @ rank n: 0.0,execution time: 0.01s
LOF ROC: 0.318, precision @ rank n: 0.0,execution time: 0.003s
MCD ROC: 0.3308, precision @ rank n: 0.0,execution time: 0.076s
OCSVM ROC: 0.4087, precision @ rank n: 0.0,execution time: 0.002s
PCA ROC: 0.3397, precision @ rank n: 0.0,execution time: 0.002s

...Processing vowels.mat ...
ABOD ROC: 0.9521, precision @ rank n: 0.4706,execution time: 0.27s
CBLOF ROC: 0.9278, precision @ rank n: 0.4118,execution time: 0.095s


c:\python36\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Feature Bagging ROC: 0.9385, precision @ rank n: 0.3529,execution time: 0.299s
HBOS ROC: 0.6758, precision @ rank n: 0.1765,execution time: 0.009s
Isolation Forest ROC: 0.7469, precision @ rank n: 0.1176,execution time: 0.5144s
KNN ROC: 0.9568, precision @ rank n: 0.5294,execution time: 0.078s
LOF ROC: 0.9345, precision @ rank n: 0.4118,execution time: 0.029s
MCD ROC: 0.6779, precision @ rank n: 0.0,execution time: 0.8046s
OCSVM ROC: 0.7415, precision @ rank n: 0.2941,execution time: 0.029s
PCA ROC: 0.5787, precision @ rank n: 0.1176,execution time: 0.004s

...Processing wbc.mat ...
ABOD ROC: 0.9232, precision @ rank n: 0.3,execution time: 0.076s
CBLOF ROC: 0.9063, precision @ rank n: 0.6,execution time: 0.048s


c:\python36\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Feature Bagging ROC: 0.9415, precision @ rank n: 0.5,execution time: 0.134s
HBOS ROC: 0.9592, precision @ rank n: 0.7,execution time: 0.008s
Isolation Forest ROC: 0.9451, precision @ rank n: 0.5,execution time: 0.357s
KNN ROC: 0.9437, precision @ rank n: 0.5,execution time: 0.04s
LOF ROC: 0.9352, precision @ rank n: 0.4,execution time: 0.016s
MCD ROC: 0.8986, precision @ rank n: 0.4,execution time: 0.14s
OCSVM ROC: 0.9408, precision @ rank n: 0.5,execution time: 0.004s
PCA ROC: 0.9324, precision @ rank n: 0.6,execution time: 0.002s


In [ ]:
outliers_fraction

In [ ]:
outliers_percentage

In [93]:
temp_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.3,0.6,0.5,0.7,0.5,0.5,0.4,0.4,0.5,0.6


In [94]:
roc_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.9181,0.9259,0.9555,0.9594,0.9355,0.9426,0.9536,0.9233,0.9433,0.9329
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,Cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9127,0.9636,0.9818
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,Cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622


In [95]:
time_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.063,0.045,0.067,0.01,0.3,0.017,0.009,0.1012,0.004,0.002
0,arrhythmia,452,274,14.6018,0.13,0.0956,0.448,0.06,0.384,0.07,0.059,1.1043,0.03,0.0588
0,Cardio,1831,21,9.6122,0.346,0.1139,0.616,0.008,0.363,0.127,0.076,0.6083,0.052,0.005
0,glass,214,9,4.2056,0.035,0.035,0.032,0.003,0.285,0.008,0.003,0.033,0.002,0.001
0,ionosphere,351,33,35.8974,0.0595,0.039,0.063,0.01,0.302,0.016,0.006,0.121,0.004,0.003
0,letter,1600,32,6.25,0.312,0.081,0.5441,0.011,0.369,0.111,0.066,1.7391,0.049,0.006
0,lympho,148,18,4.0541,0.025,0.034,0.028,0.004,0.29,0.006,0.002,0.0444,0.002,0.001
0,arrhythmia,452,274,14.6018,0.21,0.1185,0.4758,0.106,0.548,0.071,0.077,1.2781,0.03,0.061
0,Cardio,1831,21,9.6122,0.459,0.176,0.7931,0.011,0.514,0.13,0.107,0.7859,0.053,0.005
0,glass,214,9,4.2056,0.035,0.038,0.037,0.004,0.3795,0.018,0.005,0.041,0.001,0.002


In [96]:
prn_df

,Data,#Samples,#Dimesnions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.5455,0.5455,0.7273,0.6364,0.6364,0.7273,0.6364,0.5455,0.6364,0.6364
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,Cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,Cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
